In [2]:
'''
Exercise 1 (1 point). The Ising model is a basic model of statistical mechanics that explains a lot about how quantum optimizers work. 
Its energy is described by its Hamiltonian:
$$ H=-\sum_{&lt;i,j&gt;} J_{ij} \sigma_i \sigma_{j} - \sum_i h_i \sigma_i$$.
Write a function that calculates this energy amount for a linear chain of spins. 
The function takes three arguments: J, h, and σ, corresponding to the coupling strengths, the onsite field at each site, and the specific spin configuration

'''

'''
the function sums spin-spin and adds the external 
'''
def calculate_energy(J, σ,h):
    spinspin=-sum(J_ij*σ[i]*σ[i+1] for i, J_ij in enumerate(J))
    spinh =-sum(h_i*σ[i] for i, h_i in enumerate(h))
    return spinspin+spinh



J = [1.0, -1.0,-1.0, -1.0]
σ = [+1, -1, +1,+1,+1]
h = [0.5, 0.5, 0.4,0.5,-0.6]

calculate_energy(J, σ,h)

1.7

In [8]:
'''
Exercise 2 (2 points). 
The sign of the coupling defines the nature of the interaction, ferromagnetic or antiferromagnetic, corresponding to positive and negative $J$ values, respectively. 
Setting the couplings to zero, we have a non-interacting model. Create an arbitrary antiferromagnetic model on three sites with no external field. Define the model through variables J and h. Iterate over all solutions and write the optimal one in a variable called σ. 
If the optimum is degenerate, that is, you have more than one optimal configuration, keep one.

'''
import itertools
# J is negative because it is antiferromagnetic
# the itertools creates the coupling +1 and -1 for three sites (range3) 
### BEGIN SOLUTION
J = [-1, -1]
h = [0, 0, 0]
minimum_energy = 0
for σ_c in itertools.product(*[{+1,-1} for _ in range(3)]):
    energy = calculate_energy(J, h, σ_c)
    if energy < minimum_energy:
        minimum_energy = energy
        σ = σ_c
### END SOLUTION

In [9]:
σ_c


(-1, -1, -1)

In [12]:
'''
Exercise 3 (1 point). 
Iterating over all solutions is clearly not efficient, since there are exponentially many configurations in the number of sites. 
From the perspective of computer science, this is a combinatorial optimization problem, and it is a known NP-hard problem. 
Many heuristic methods have been invented to tackle the problem. One of them is simulated annealing. 
It is implemented in dimod. Create the same antiferromagnetic model in dimod as above. 
Keep in mind that dimod uses a plus and not a minus sign in the Hamiltonian, so the sign of your couplings should be reversed. 
Store the model in an object called model, which should be a BinaryQuadraticModel.

'''
# we use the dimod from dwave
# the coupling J is positive
J = {(0, 1): 1.0, (1, 2): 1.0} # two Js means three spins
h = {0:0, 1:0, 2:0} # no external interaction
import dimod
model = dimod.BinaryQuadraticModel(h, J, 0.0, dimod.SPIN)
sampler = dimod.SimulatedAnnealingSampler()
response = sampler.sample(model, num_reads=100)

In [13]:
response

SampleSet(rec.array([([-1,  1, -1], -2., 1), ([-1,  1, -1], -2., 1),
           ([ 1, -1,  1], -2., 1), ([-1,  1, -1], -2., 1),
           ([ 1, -1,  1], -2., 1), ([-1,  1, -1], -2., 1),
           ([-1,  1, -1], -2., 1), ([-1,  1, -1], -2., 1),
           ([-1,  1, -1], -2., 1), ([-1,  1, -1], -2., 1),
           ([ 1, -1,  1], -2., 1), ([-1,  1, -1], -2., 1),
           ([ 1, -1,  1], -2., 1), ([-1,  1, -1], -2., 1),
           ([-1,  1, -1], -2., 1), ([-1,  1, -1], -2., 1),
           ([-1,  1, -1], -2., 1), ([ 1, -1,  1], -2., 1),
           ([-1,  1, -1], -2., 1), ([-1,  1, -1], -2., 1),
           ([ 1, -1,  1], -2., 1), ([ 1, -1,  1], -2., 1),
           ([-1,  1, -1], -2., 1), ([-1,  1, -1], -2., 1),
           ([ 1, -1,  1], -2., 1), ([-1,  1, -1], -2., 1),
           ([-1,  1, -1], -2., 1), ([ 1, -1,  1], -2., 1),
           ([-1,  1, -1], -2., 1), ([ 1, -1,  1], -2., 1),
           ([ 1, -1,  1], -2., 1), ([ 1, -1,  1], -2., 1),
           ([ 1, -1,  1], -2., 1), ([ 1, -1,  

In [14]:
assert len(response) == 100, "Not the correct number of samples"
sample = response.first.sample
assert all([sample[i]*sample[i+1] == -1 for i, _ in enumerate(J.values())]), "The optimal configuration is not antiferromagnetic"

In [29]:
'''

The transverse-field Ising model

Exercise 5 (1 point). Adiabatic quantum computation and quantum annealing rely on quantum variants of the classical Ising model, 
and so do some variational algorithms like the quantum approximate optimization algorithm. 
To understand the logic behind these simple quantum-many body systems, first let us take another look at the classical Ising model, but write the 
Hamiltonian of the system in the quantum mechanical formalism, that is, with operators:
$$ H=-\sum_{&lt;i,j&gt;} J_{ij} \sigma^Z_i \sigma^Z_{j} - \sum_i h_i \sigma^Z_i$$

'''

# Z is the pauli at site 1
import numpy as np
Z1=np.array([[1,0],[0,-1]])
Z1
Z2=np.array([[1,0],[0,-1]])
# the Hamiltonian is Z1 tensor Z2
H=-np.kron(Z1,Z2)
print(H)
# get eigenvalues


_, eigenvectors = np.linalg.eigh(H)
print(eigenvectors[:, 0:1])
print(eigenvectors[:, 1:2])
print(eigenvectors)



[[-1  0  0  0]
 [ 0  1  0  0]
 [ 0  0  1  0]
 [ 0  0  0 -1]]
[[1.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [1.]]
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]]


In [19]:
H

array([[ 1,  0,  0,  0],
       [ 0, -1,  0,  0],
       [ 0,  0, -1,  0],
       [ 0,  0,  0,  1]])

In [31]:
values, eigenvectors = np.linalg.eigh(H)
'''
print(eigenvectors[:, 0:1])
print(eigenvectors[:, 1:2])
'''
print(values,eigenvectors)
'''
we have two eigenvalues +1 and -1. The 00 and 11 states (aligned and antialigned) correspond

the gound state is 1,0,0,0 or 0,0,0,1
we want to calculate <00|H|00>
'''
def calculate_energy_expectation(state, hamiltonian):
    return float(np.dot(state.T.conj(), np.dot(hamiltonian, state)).real)

ψ = np.kron([[0], [1]], [[0], [1]])
calculate_energy_expectation(ψ, H)

[-1. -1.  1.  1.] [[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]]


-1.0

In [40]:
'''
What is we add an X to the H?
Since we have two qubits, we have I_1X_2 and X_1I_2
'''

# basic Hamiltonian
Z1=np.array([[1,0],[0,-1]])
Z1
Z2=np.array([[1,0],[0,-1]])
# the Hamiltonian is Z1 tensor Z2
H_1=-np.kron(Z1,Z2)
print(H_1)

X = np.array([[0, 1], [1, 0]])
IX = np.kron(np.eye(2), X)
XI = np.kron(X, np.eye(2))
H = H_1 -XI-IX
print(H)

# the X change the eigenvectors
values, eigenvectors = np.linalg.eigh(H)
for i in range(len(values)):
    print(values[i], eigenvectors[i])
    print(calculate_energy_expectation(eigenvectors[i], H))

[[-1  0  0  0]
 [ 0  1  0  0]
 [ 0  0  1  0]
 [ 0  0  0 -1]]
[[-1. -1. -1.  0.]
 [-1.  1.  0. -1.]
 [-1.  0.  1. -1.]
 [ 0. -1. -1. -1.]]
-2.23606797749979 [ 6.01500955e-01 -7.07106781e-01 -1.11022302e-16  3.71748034e-01]
1.3763819204711731
-0.9999999999999999 [ 3.71748034e-01  4.29322157e-18  7.07106781e-01 -6.01500955e-01]
0.3249196962329064
1.0 [ 3.71748034e-01  4.29322157e-18 -7.07106781e-01 -6.01500955e-01]
-0.3249196962329065
2.236067977499789 [ 6.01500955e-01  7.07106781e-01 -2.22044605e-16  3.71748034e-01]
-1.3763819204711727
